Zde hledáme Wikidata IDs toho autorstva, které je nemá vyplněné v databázi autorit.

In [6]:
import gc
import os
import re
import json
import requests
import pandas as pd

In [7]:
kam = 'data_raw/wikidata_doplneni'
if not os.path.exists(kam):
    os.makedirs(kam)

In [29]:
vsechny_jsony = [f for f in os.listdir(kam)]

In [31]:
vyhledane = set([x.split("_")[0] for x in vsechny_jsony])

In [10]:
df = pd.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))

In [11]:
df = df.explode('100_7')

In [12]:
df['100_7'].nunique()

171303

In [13]:
df.groupby('100_7').size().nlargest(1000)

100_7
jx20060515016    2960
jx20050830020    2373
jn20001103529    1652
jn19981000826    1468
jk01120043       1385
                 ... 
jn19990209875      65
jn19990210374      65
jn20000710062      65
jn20000810185      65
jn99240001194      65
Length: 1000, dtype: int64

In [14]:
opakovane = df.groupby('100_7').size()
opakovane = opakovane.sort_values(ascending=False)
opakovane = opakovane[opakovane > 1].index.to_list()
len(opakovane)

90630

In [15]:
autority = pd.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [16]:
autority

,024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,...,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader
001,,,,,,,,,,,,,,,,,,,,,
jk01010001,None,None,[1931],None,[jk01010001],"[Abraham, Jiří,]",[1931-],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 14.12.1931 v Osvračíně. MUDr., odborn...",None,nz a22 n 4500
jk01010002,None,None,[1890],[1933],[jk01010002],"[Abraham, Josef,]",[1890-1933],[1],None,None,...,None,None,None,None,None,None,None,[Narozen 19.11.1890 v Osvračíně u Horšova Týna...,None,nz a22 n 4500
jk01010003,None,None,[1957],None,[jk01010003],"[Abraham, Pavel,]",[1957-],[1],None,None,...,None,None,None,None,None,None,None,[Narozen roku 1957. Student filosofické fakult...,None,nz a22 n 4500
jk01010004,None,None,[1889],[1963],[jk01010004],"[Absolon, Josef,]",[1889-1963],[1],None,"[Odrlice, Senice na Hané, Česko]",...,None,None,None,None,None,None,None,"[Narozen 27. 1. 1889 v Odrlicích na Moravě, ze...",None,cz a22 n 4500
jk01010005,"[isni, wikidata]","[0000000109542912, Q1063371]",[1877],[1960],[jk01010005],"[Absolon, Karel,]",[1877-1960],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 16.6.1877 v Boskovicích, zemřel 6.10....",None,cz a22 n 4500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xx0323367,None,None,None,None,[xx0323367],"[Birhanu, Abigail]",None,[1],None,None,...,None,None,None,None,None,None,None,[Americká výtvarnice a učitelka výtvarné výcho...,None,nz a22 n 4500
xx0323368,None,None,None,None,[xx0323368],"[Latimer, Khadesia]",None,[1],None,None,...,None,None,None,None,None,None,None,[Americká výtvarnice a výtvarná pedagožka zamě...,None,nz a22 n 4500
xx0323369,None,None,None,None,[xx0323369],"[Liz, Paula]",None,[1],None,None,...,None,None,None,None,None,None,None,[Americká výtvarnice a výtvarná pedagožka.],None,nz a22 n 4500


In [17]:
autority = autority[autority['024_2'].notnull()]
autority = autority[autority['024_a'].notnull()]

In [18]:
autority = autority.explode(['024_a','024_2'])

In [19]:
autority_s_wikidaty = set(autority[autority['024_2'] == 'wikidata']['024_a'].index.to_list())

In [20]:
chybi = [x for x in opakovane if x not in autority_s_wikidaty]

In [21]:
del df
del autority
gc.collect()

0

In [22]:
len(chybi)

82368

In [23]:
# Wikidata SPARQL endpoint
sparql_endpoint = "https://query.wikidata.org/sparql"

# Function to get Wikidata ID from NL CR AUT ID
def get_wikidata_id(nl_cr_aut_id):
    query = f"""
    SELECT ?item WHERE {{
      ?item wdt:P691 "{nl_cr_aut_id}".
    }}
    """
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(sparql_endpoint, params={"query": query}, headers=headers)
    data = response.json()

    if "results" in data and "bindings" in data["results"]:
        bindings = data["results"]["bindings"]
        if bindings:
            return bindings[0]["item"]["value"].split("/")[-1]
    return None

In [24]:
vyhledane = set(vyhledane)

In [33]:
co_stahovat = [x for x in chybi if x not in vyhledane]

In [35]:
len(co_stahovat)

19918

In [37]:
pocitadlo = 0
for nl_cr_aut_id in co_stahovat:
    pocitadlo += 1
    print(f"{pocitadlo:005}: {nl_cr_aut_id}")
    slovnik = {}
    slovnik['100_7'] = nl_cr_aut_id
    wikidata_id = get_wikidata_id(nl_cr_aut_id)
    if wikidata_id:
        slovnik['024_a'] = wikidata_id
    else:
        slovnik['024_a'] = ""
    with open(os.path.join(kam, f"{nl_cr_aut_id}_{slovnik['024_a']}.json"), "w+", encoding="utf-8") as file:
        file.write(json.dumps(slovnik))

00001: jk01110197
00002: mzk2006368435
00003: jk01110129
00004: jk01110124
00005: xx0000807
00006: mzk2015868139
00007: xx0227842
00008: jk01110077
00009: mzk2008486278
00010: jk01042226
00011: xx0019185
00012: xx0227805
00013: mzk2017970057
00014: mzk2007271855
00015: mzk2017970078
00016: mzk2007271852
00017: xx0019200
00018: jk01090143
00019: mzk2007265155
00020: mzk2007265154
00021: jk01110097
00022: mzk2017970661
00023: xx0012614
00024: mzk2006377744
00025: xx0019204
00026: mzk2017970900
00027: mzk2006377735
00028: jk01110107
00029: mzk2006377728
00030: xx0019212
00031: jk01110114
00032: mzk2007377397
00033: jk01042262
00034: jk01103393
00035: mzk2007377580
00036: jk01103203
00037: mzk2017964886
00038: jk01042340
00039: mzk2007377559
00040: xx0012694
00041: mzk2008479794
00042: xx0018888
00043: xx0018889
00044: xx0000874
00045: mzk2015869233
00046: xx0018923
00047: xx0012677
00048: mzk2008479798
00049: xx0000869
00050: mzk2008479816
00051: mzk2008479819
00052: xx0012662
00053: mzk2